In [ ]:
import sys
import os
sys.path.append('../../')

from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
from sklearn.model_selection import train_test_split

from rcalgo.tf.tfmodels.dl_text_classification import *
from rcalgo.tf.utils.text_utils import *

import pandas as pd 
import numpy as np


In [ ]:
model_input_dir = '../sentence-embedding/model_out'
batch_size = 256
model_name = 'bi_transformer_2l_photo_comment_3'
job = IncrementalTraining(model_input_dir, model_name)

inp_text = "我想和你在一起，你知不知道我爱你"
out_text = inp_text
layer = 2
with job.graph.as_default():
    inputs = tf.get_collection('input_dict')[0]
    fw_attns = []
    bw_attns = []
    for i in range(layer):
        if i == 0:
            fw_attn = tf.get_default_graph().get_tensor_by_name('Tower_0/gpu/bidirectional_transformer/transformer_fw/transform2_block/multi_head_attetion/cond/Merge:0')
            bw_attn = tf.get_default_graph().get_tensor_by_name('Tower_0/gpu/bidirectional_transformer/transformer_bw/transform2_block/multi_head_attetion/cond/Merge:0')
        else:
            fw_attn = tf.get_default_graph().get_tensor_by_name('Tower_0/gpu/bidirectional_transformer/transformer_fw/transform2_block_{}/multi_head_attetion/cond/Merge:0'.format(i))
            bw_attn = tf.get_default_graph().get_tensor_by_name('Tower_0/gpu/bidirectional_transformer/transformer_bw/transform2_block/multi_head_attetion/cond/Merge:0')

        fw_attns.append(fw_attn)
        bw_attns.append(bw_attn)
    fw_attn_py = job.session.run(fw_attns, feed_dict={inputs:[inp_text]})
    bw_attn_py = job.session.run(bw_attns, feed_dict={inputs:[inp_text]})
    
fw_attn_py = np.stack(fw_attn_py, axis=0)
bw_attn_py = np.stack(bw_attn_py, axis=0)

fw_attn_py = fw_attn_py[:, :, :, 0:len(inp_text), 0:len(inp_text)]
bw_attn_py = bw_attn_py[:, :, :, 0:len(inp_text), 0:len(inp_text)]

In [ ]:
def _get_attention(inp_text, out_text, enc_atts):

    def get_inp_inp_attention(layer):
        att = np.transpose(enc_atts[layer][0], (0, 2, 1))
        return [ha.T.tolist() for ha in att]

    def get_attentions(get_attention_fn):
        num_layers = len(enc_atts)
        attentions = []
        for i in range(num_layers):
            attentions.append(get_attention_fn(i))

        return attentions

    attentions = {
        'inp_inp': {
            'att': get_attentions(get_inp_inp_attention),
            'top_text': inp_text,
            'bot_text': inp_text,
        }
    }

    return attentions

In [ ]:
fw_att =_get_attention(
      inp_text, out_text, fw_attn_py)
bw_att =_get_attention(
      inp_text, out_text, bw_attn_py)
all_att = _get_attention(inp_text, out_text, fw_attn_py + bw_attn_py)
fw_att_json = json.dumps(fw_att)
bw_att_json = json.dumps(bw_att)
all_att_json = json.dumps(all_att)

In [ ]:
vis_js = open('attention.js').read()
import IPython.display as display
vis_html = """
  <span style="user-select:none">
    Layer: <select id="layer"></select>
    Attention: <select id="att_type">
      <option value="inp_inp">Input - Input</option>
    </select>
  </span>
  <div id='vis'></div>
"""

In [ ]:
import IPython.display as display
def _show_attention(att_json):
    display.display(display.HTML(vis_html))
    display.display(display.Javascript('window.attention = {}, window.layer= {}'.format(att_json, layer)))
    display.display(display.Javascript(vis_js))
_show_attention(fw_att_json)
# _show_attention(bw_att_json)
# _show_attention(all_att_json)